# Evaluation of results from the simulations for the different scenarios
1. Reserved bandwidth for the different services at the different nodes
2. Latencies of the different services at different receivers
3. Queue lengths of the different priorities at the different nodes

In [69]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline  

In [70]:
# # read the omnetpp csv results into panda data frames.
# reservation = pd.read_csv('reservedBandwidth.csv')
# latencies = pd.read_csv('serviceLatencies.csv')
# queues = pd.read_csv('allQueueLengths.csv')

## Definitions

## Frequently used functions

In [71]:
def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def parseCSV(csvFile):
    return pd.read_csv(csvFile, converters = {
        'attrvalue': parse_if_number,
        'binedges': parse_ndarray,
        'binvalues': parse_ndarray,
        'vectime': parse_ndarray,
        'vecvalue': parse_ndarray})

def moduleIdxIsNaN(data, idx):
    return data['module'][idx] is np.nan

def idxHasVecVal(data, idx):
    return data['vecvalue'][idx] is not None

def getVecValAndTime(data, i, j):
    return data['vecvalue'][i][j], data['vectime'][i][j]

def isModuleSwitchPort(module):
    return '.S[' in module and '.etherMAC[' in module

def isModuleNode(module):
    return '.N[' in module

def isModuleService(module):
    return '.services[' in module

def removeNetworkFromModule(module):
    # remove everything before the first dot
    return module[module.find('.')+1:]

def isStatReservedBandwidth(stat):
    return 'reservedBandwidth' in stat

def isStatServiceLatency(stat):
    return 'rxLatency' in stat

def isStatCredit(stat):
    return 'credit' in stat

def getRunName(run):
    return run.split('-')[0]

def getParamForModule(data, module, param):
    moduleParamPath = "**." + removeNetworkFromModule(module) + "." + param
    # filter data for 'type' = param
    for i in range(0, len(data)):
        if data['type'][i] == "config" and data['attrname'][i] == moduleParamPath:
            return data['attrvalue'][i]
    return None

def findServiceIdForModule(data, module):
    return int(getParamForModule(data, module, "serviceId"))
    
def parseModuleNameQueue(module):
    if isModuleNode(module):
        nodeNo = int(re.search(r'N\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'queue\[(.*?)\]', module).group(1))
        return "N[{}] Queue[{}]".format(nodeNo, cbsPrio)
    elif isModuleSwitchPort(module):
        switchNo = int(re.search(r'S\[(.*?)\]', module).group(1))
        portNo = int(re.search(r'etherMAC\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'queue\[(.*?)\]', module).group(1))
        return "S[{}] Port[{}] Queue[{}]".format(switchNo, portNo, cbsPrio)
    return None
    
def parseModuleNameShaper(module):
    if isModuleNode(module):
        return parseModuleNameNodeCBS(module)
    elif isModuleSwitchPort(module):
        return parseModuleNameSwitchCBS(module)
    return None
    
def parseModuleNameSwitchCBS(module):
    # extract switch and algorithm number from SomeIp_Qos_Small.S[0].etherMAC[0].shaper.transmissionSelectionAlgorithm[5] 
    if isModuleSwitchPort(module):
        switchNo = int(re.search(r'S\[(.*?)\]', module).group(1))
        portNo = int(re.search(r'etherMAC\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'Algorithm\[(.*?)\]', module).group(1))
        return "S[{}] Port[{}] Queue[{}] CBS".format(switchNo, portNo, cbsPrio)
    return None

def parseModuleNameNodeCBS(module):
    # extract node and algorithm number from SomeIp_Qos_Small.N[0].shaper.transmissionSelectionAlgorithm[5] 
    if isModuleNode(module):
        nodeNo = int(re.search(r'N\[(.*?)\]', module).group(1))
        cbsPrio = int(re.search(r'Algorithm\[(.*?)\]', module).group(1))
        return "N[{}] Queue[{}] CBS".format(nodeNo, cbsPrio)
    return None

def parseNodeName(module):
    if isModuleNode(module):
        nodeNo = int(re.search(r'N\[(.*?)\]', module).group(1))
        return "N[{}]".format(nodeNo)

def parseServiceName(data, module):
    if isModuleService(module):
        serviceId = findServiceIdForModule(data, module)
        return "ServiceId {}".format(serviceId)
    return None


## 1. Reserved bandwidth for the different services at the different nodes

In [72]:
# read the omnetpp csv results into a panda data frame
reservation = parseCSV('reservedBandwidth.csv')
reservation.head()   

,run,type,module,name,attrname,attrvalue,vectime,vecvalue
0,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,configname,SmallEvalCase_FixedCMI_NoSourceCT,None,None
1,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetime,20240221-11:22:03,None,None
2,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetimef,20240221-112203,None,None
3,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,experiment,SmallEvalCase_FixedCMI_NoSourceCT,None,None
4,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,inifile,omnetpp.ini,None,None


In [73]:
# iterate over the modules find the switches (S[*]) and print their reserved bandwidth
runCBSReservation = dict()
for i in range(0, len(reservation)):
    if moduleIdxIsNaN(reservation, i):
        continue
    if isModuleSwitchPort(reservation['module'][i]):
        if isStatReservedBandwidth(reservation['name'][i]):
            # filter reservation['vecvalue'][i] for non zero values and print them
            if not idxHasVecVal(reservation, i):
                continue
            for j in range(0, len(reservation['vecvalue'][i])):
                val, time = getVecValAndTime(reservation, i, j)
                if val != 0:
                    # insert values into switchBandwidth dict
                    runName = getRunName(reservation['run'][i])
                    if runName not in runCBSReservation:
                        runCBSReservation[runName] = dict()
                    switchName = parseModuleNameSwitchCBS(reservation['module'][i])
                    if switchName not in runCBSReservation[runName]:
                        runCBSReservation[runName][switchName] = dict()
                    runCBSReservation[runName][switchName][time] = val
runCBSReservation


{'SmallEvalCase_FixedCMI_NoSourceCT': {'S[1] Port[1] Queue[7] CBS': {0.02076849: 98240000.0},
  'S[0] Port[0] Queue[7] CBS': {0.02076849: 98240000.0},
  'S[2] Port[1] Queue[7] CBS': {0.02076849: 98240000.0}},
 'SmallEvalCase_NoSourceCT': {'S[1] Port[1] Queue[7] CBS': {0.02076849: 12288000.0},
  'S[0] Port[0] Queue[7] CBS': {0.02076849: 12288000.0},
  'S[2] Port[1] Queue[7] CBS': {0.02076849: 12288000.0}}}

In [74]:
# output the switchBandwidth dict to csv
for run in runCBSReservation.keys():
    switchCBSBandwidthDF = pd.DataFrame.from_dict(runCBSReservation[run])
    # sord inner dict by key
    switchCBSBandwidthDF = switchCBSBandwidthDF.apply(lambda x: x.sort_index())
    switchCBSBandwidthDF.to_csv("export/bandwidth"+run+".csv", index=True, header=True, sep=';')

In [75]:
# also create one common csv file for all runs
switchCBSBandwidthDF = pd.DataFrame.from_dict(runCBSReservation)
# sord inner dict by key
switchCBSBandwidthDF = switchCBSBandwidthDF.apply(lambda x: x.sort_index())
switchCBSBandwidthDF.to_csv("export/bandwidth.csv", index=True, header=True, sep=';')

## 2. Latencies of the different services at different receivers

In [76]:
# read the omnetpp csv results into a panda data frame
latencies = parseCSV('latency.csv')
latencies.head()   


,run,type,module,name,attrname,attrvalue,vectime,vecvalue
0,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,configname,SmallEvalCase_FixedCMI_NoSourceCT,None,None
1,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetime,20240221-11:22:03,None,None
2,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetimef,20240221-112203,None,None
3,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,experiment,SmallEvalCase_FixedCMI_NoSourceCT,None,None
4,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,inifile,omnetpp.ini,None,None


In [77]:
runLatencies = dict()
for i in range(0, len(latencies)):
    if moduleIdxIsNaN(latencies, i):
        continue
    if isModuleService(latencies['module'][i]):
        if isStatServiceLatency(latencies['name'][i]):
            if not idxHasVecVal(latencies, i):
                continue
            runName = getRunName(latencies['run'][i])
            if runName not in runLatencies:
                runLatencies[runName] = dict()
            node = parseNodeName(latencies['module'][i])
            if node not in runLatencies[runName]:
                runLatencies[runName][node] = dict()
            service = parseServiceName(latencies, latencies['module'][i])
            if service not in runLatencies[runName][node]:
                runLatencies[runName][node][service] = dict()
            for j in range(0, len(latencies['vecvalue'][i])):
                val, time = getVecValAndTime(latencies, i, j)
                runLatencies[runName][node][service][time] = val
runLatencies

{'SmallEvalCase_FixedCMI_NoSourceCT': {'N[2]': {'ServiceId 0': {0.02151124: 0.00051124,
    0.02287806: 0.00087806,
    0.023598986509: 0.000598986509,
    0.02451124: 0.00051124,
    0.025617700512: 0.000617700512,
    0.026594363317: 0.000594363317,
    0.027595149356: 0.000595149356,
    0.028598185962: 0.000598185962,
    0.029530255801: 0.000530255801,
    0.030560385148: 0.000560385148,
    0.03151124: 0.00051124,
    0.032579930693: 0.000579930693,
    0.03357967009: 0.00057967009,
    0.03468671009: 0.00068671009,
    0.035553173172: 0.000553173172,
    0.036634456002: 0.000634456002,
    0.03751124: 0.00051124,
    0.038591511187: 0.000591511187,
    0.03951124: 0.00051124,
    0.040630901344: 0.000630901344,
    0.041585459878: 0.000585459878,
    0.042630747525: 0.000630747525,
    0.043568128094: 0.000568128094,
    0.044584401826: 0.000584401826,
    0.045586257647: 0.000586257647,
    0.04651124: 0.00051124,
    0.047555209191: 0.000555209191,
    0.048628407511: 0.000628

In [78]:
# plot the latencies one graph per run containing all nodes and services
for run in runLatencies.keys():
    fig, ax = plt.subplots()
    for node in runLatencies[run].keys():
        for service in runLatencies[run][node].keys():
            x = runLatencies[run][node][service].keys()
            y = runLatencies[run][node][service].values()
            ax.plot(x, y, label=node + " " + service)
    ax.legend()
    ax.set_xlabel('time')
    ax.set_ylabel('latency [s]')
    ax.set_title('Latencies for run ' + run)
    ax.grid(True)
    ax.set_xlim(0, 2)  # set x-axis limits to 0-100ms
    ax.set_ylim(0, 0.001)  # show latencies below 1ms
    fig.savefig("export/latencies"+run+".png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [79]:
# plot the latencies one graph per service containing all nodes for all runs
tmpFigs = dict()
tmpAxs = dict()
deadlines = {
    "ServiceId 1 N[3]":0.001,
    "ServiceId 3 N[3]":0.005,
    "ServiceId 0 N[2]":0.001,
    "ServiceId 3 N[2]":0.005,
    "ServiceId 2 N[1]":0.010
}
for run in runLatencies.keys():
    for node in runLatencies[run].keys():
        for service in runLatencies[run][node].keys():
            key = service + " " + node
            if key not in tmpFigs:
                tmpFigs[key], tmpAxs[key] = plt.subplots()
            x = runLatencies[run][node][service].keys()
            y = runLatencies[run][node][service].values()
            tmpAxs[key].plot(x, y, label=run, linewidth=0.5, linestyle='solid')
for name in tmpFigs.keys():
    # add deadline for the name to the plot
    tmpAxs[name].axhline(y=deadlines[name], color='r', linestyle='-', label="deadline")
    tmpAxs[name].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3)
    tmpAxs[name].set_xlabel('time')
    tmpAxs[name].set_ylabel('latency [s]')
    tmpAxs[name].set_title('Latencies for ' + name)
    tmpAxs[name].grid(True)
    tmpAxs[name].set_xlim(0, 0.1)  # set 
    tmpAxs[name].set_ylim(0, deadlines[name] + 0.0001)  # show latencies below 1ms
    tmpFigs[name].savefig("export/latencies"+name+".png", dpi=300, bbox_inches='tight')
    plt.close(tmpFigs[name])

## 3. Queue lengths of the different priorities at the different nodes

In [80]:
# read the omnetpp csv results into a panda data frame
queues = parseCSV('allQueueLengths.csv')
queues.head()

FileNotFoundError: [Errno 2] No such file or directory: 'allQueueLengths.csv'

In [ ]:
# determine min max and average queue lengths for each run, node port and queue
runQueueLengths = dict()
for i in range(0, len(queues)):
    if moduleIdxIsNaN(queues, i):
        continue
    moduleName = parseModuleNameQueue(queues['module'][i])
    if moduleName is None or not idxHasVecVal(queues, i):
        continue
    runName = getRunName(queues['run'][i])
    if runName not in runQueueLengths:
        runQueueLengths[runName] = dict()
    if moduleName not in runQueueLengths[runName]:
        runQueueLengths[runName][moduleName] = dict()
    runQueueLengths[runName][moduleName]['min'] = min(queues['vecvalue'][i])
    runQueueLengths[runName][moduleName]['max'] = max(queues['vecvalue'][i])
    runQueueLengths[runName][moduleName]['avg'] = sum(queues['vecvalue'][i])/len(queues['vecvalue'][i])

In [ ]:
# plot the min max avg queue lengths as boxes one graph per run containing all nodes and services
for run in runQueueLengths.keys():
    fig, ax = plt.subplots()
    for node in runQueueLengths[run].keys():
        ax.plot([node, node], [runQueueLengths[run][node]['min'], runQueueLengths[run][node]['max']], solid_capstyle="butt")
        ax.plot(node, [runQueueLengths[run][node]['avg']], marker="o")
    ax.set_ylabel('queue length [#packets]')
    ax.set_title('Queue lengths for run ' + run)
    ax.grid(True)
    ax.tick_params(axis='x', rotation=90)
    fig.savefig("export/queues"+run+".png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [ ]:
# plot the min max avg queue lengths as boxes one graph per node containing all runs
tmpFigs = dict()
tmpAxs = dict()
for run in runQueueLengths.keys():
    for node in runQueueLengths[run].keys():
        key = node
        if key not in tmpFigs:
            tmpFigs[key], tmpAxs[key] = plt.subplots()
        tmpAxs[key].plot([run, run], [runQueueLengths[run][node]['min'], runQueueLengths[run][node]['max']], solid_capstyle="butt")
        tmpAxs[key].plot(run, [runQueueLengths[run][node]['avg']], marker="o")
for name in tmpFigs.keys():
    tmpAxs[name].set_ylabel('queue length [#packets]')
    tmpAxs[name].set_title('Queue lengths for ' + name)
    tmpAxs[name].grid(True)
    tmpAxs[name].tick_params(axis='x', rotation=90)
    tmpFigs[name].savefig("export/queues"+name+".png", dpi=300, bbox_inches='tight')
    plt.close(tmpFigs[name])    

C:\Users\Timo\AppData\Local\Temp\ipykernel_12584\2729343739.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  tmpFigs[key], tmpAxs[key] = plt.subplots()


## 4. Credit Vector

In [81]:
# read the omnetpp csv results into a panda data frame
credit = parseCSV('credit.csv')
credit.head()   

,run,type,module,name,attrname,attrvalue,vectime,vecvalue
0,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,configname,SmallEvalCase_FixedCMI_NoSourceCT,None,None
1,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetime,20240221-11:22:03,None,None
2,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,datetimef,20240221-112203,None,None
3,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,experiment,SmallEvalCase_FixedCMI_NoSourceCT,None,None
4,SmallEvalCase_FixedCMI_NoSourceCT-0-20240221-1...,runattr,NaN,NaN,inifile,omnetpp.ini,None,None


In [82]:
credits = dict()
for i in range(0, len(credit)):
    if moduleIdxIsNaN(credit, i):
        continue
    if not idxHasVecVal(credit, i):
        continue
    if not isStatCredit(credit['name'][i]):
        continue
    runName = getRunName(credit['run'][i])
    if runName not in credits:
        credits[runName] = dict()
    node = parseModuleNameShaper(credit['module'][i])
    if node is None:
        continue
    if node not in credits[runName]:
        credits[runName][node] = dict()
    for j in range(0, len(credit['vecvalue'][i])):
        val, time = getVecValAndTime(credit, i, j)
        credits[runName][node][time] = val
credits

    

{'SmallEvalCase_FixedCMI_NoSourceCT': {'S[2] Port[1] Queue[7] CBS': {0.02080963: 0.0,
   0.02123749: 0.0,
   0.02138943: 0.0,
   0.021512146932: -216.0,
   0.02151215: -216.0,
   0.021514386876: 0.0,
   0.02251017: 0.0,
   0.02263321: 0.0,
   0.02263402: 0.0,
   0.02275625: 12001.0,
   0.022878966932: 11785.0,
   0.02287897: 11785.0,
   0.02300201: 0.0,
   0.02312505: 0.0,
   0.02324809: 0.0,
   0.023476856509: 0.0,
   0.023477176509: 32.0,
   0.023599893441: -184.0,
   0.023599896509: -184.0,
   0.023601813393: 0.0,
   0.023722936509: 0.0,
   0.023845976509: 0.0,
   0.024103564969: 0.0,
   0.024226604969: 0.0,
   0.024349644969: 0.0,
   0.02438943: 0.0,
   0.024512146932: -216.0,
   0.02451215: -216.0,
   0.024514386876: 0.0,
   0.02463519: 0.0,
   0.02475823: 0.0,
   0.02488127: 0.0,
   0.02508069411: 0.0,
   0.02523764848: 0.0,
   0.02536068848: 0.0,
   0.02538943: 0.0,
   0.025495890512: 10461.0,
   0.025618607444: 10245.0,
   0.025618610512: 10245.0,
   0.025744524026: 0.0,
   0.0

In [83]:
# plot credits one graph per run containing all node and switche ports
for run in credits.keys():
    fig, ax = plt.subplots()
    for node in credits[run].keys():
        x = credits[run][node].keys()
        y = credits[run][node].values()
        ax.plot(x, y, label=node)
    ax.legend()
    ax.set_xlabel('time')
    ax.set_ylabel('credit [bytes]')
    ax.set_title('Credits for run ' + run)
    ax.grid(True)
    ax.set_xlim(0, 2)  # set x-axis limits to 0-100ms
    fig.savefig("export/credits"+run+".png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [84]:
# plot credits one graph per node containing all runs
tmpFigs = dict()
tmpAxs = dict()
for run in credits.keys():
    for node in credits[run].keys():
        key = node
        if key not in tmpFigs:
            tmpFigs[key], tmpAxs[key] = plt.subplots()
        x = credits[run][node].keys()
        y = credits[run][node].values()
        tmpAxs[key].plot(x, y, label=run)
for name in tmpFigs.keys():
    tmpAxs[name].legend()
    tmpAxs[name].set_xlabel('time')
    tmpAxs[name].set_ylabel('credit [bytes]')
    tmpAxs[name].set_title('Credits for ' + name)
    tmpAxs[name].grid(True)
    tmpAxs[name].set_xlim(0, 2)  # set x-axis limits to 0-100ms
    tmpFigs[name].savefig("export/credits"+name+".png", dpi=300, bbox_inches='tight')
    plt.close(tmpFigs[name])

In [85]:
# plot credits one graph per node and run
for run in credits.keys():
    for node in credits[run].keys():
        fig, ax = plt.subplots()
        x = credits[run][node].keys()
        y = credits[run][node].values()
        ax.plot(x, y)
        ax.set_xlabel('time')
        ax.set_ylabel('credit [bytes]')
        ax.set_title('Credits for ' + node + " in run " + run)
        ax.grid(True)
        ax.set_xlim(0, 2)  # set x-axis limits to 0-100ms
        fig.savefig("export/credits"+node+run+".png", dpi=300, bbox_inches='tight')
        plt.close(fig)